Segmenting and Clustering Neighborhoods in Toronto

In [1]:
# import necessary libraries
from bs4 import BeautifulSoup
import requests # Requests allows you to send organic, grass-fed HTTP/1.1 requests, without the need for manual labor.
from urllib import request, response, error, parse
from urllib.request import urlopen
import pandas as pd

In [2]:
# Load article, turn into soup and check the title to confirm it
website_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html =urlopen(website_url)
soup = BeautifulSoup(html, "lxml")
title = soup.title
titleText = title.get_text ()
print(titleText)


List of postal codes of Canada: M - Wikipedia


In [3]:
# find class ‘wikitable sortable’ in the HTML script
My_table = soup.find('table',{'class':'wikitable sortable'})

In [4]:
# Search through the tables for the one with the headings we want.
for table in My_table:
    ths = My_table.findAll('th')
    headings = [th.text.strip() for th in ths]
    headings[:3] == ['Postcode', 'Borough', 'Neighbourhood']

In [5]:
# Extract the columns we want and write to a comma-delimited file.
with open('output.csv', 'w') as fo:
    for tr in My_table.findAll('tr'):
        tds = tr.findAll('td')
        if not tds:
            continue
        Postcode, Borough, Neighbourhood = [td.text.strip() for td in tds[:3]]
        print(', '.join([Postcode, Borough, Neighbourhood]), file=fo)

In [6]:
#Convert the list into Pandas DataFrame to work in python
import pandas as pd
df = pd.read_csv('output.csv', header=None, names=['PostalCode', 'Borough', 'Neighbourhood'])
df

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Etobicoke,Islington Avenue


In [7]:
#Drop a row if it contains 'Not assigned' 
df[df.Borough != 'Not assigned']

df = df.dropna()
null = 'Not assigned'
df = df[(df.PostalCode != null) & (df.Borough != null) & (df.Neighbourhood != null)]

In [8]:
df.shape

(287, 3)

In [9]:
def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighbourhood'].tolist()))
                    
grp = df.groupby(['PostalCode', 'Borough'])
df2 = grp.apply(neighborhood_list).reset_index(name='Neighbourhood')

In [ ]:
import numpy as np # library to handle data in a vectorized manner
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library



Solving environment: | 

# Part II- Add Latitude and Longitude

In [20]:
!conda install -c conda-forge geocoder --yes 
import geocoder # import geocoder

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          59 KB

The following NEW packages will be INSTALLED:

    geocoder: 1.38.1-py_1 conda-forge
    ratelim:  0.1.6-py_2  conda-forge


geocoder-1.38.1      | 53 KB     | ##################################### | 100% 
ratelim-0.1.6        | 6 KB      | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [21]:

def get_latlng(postal_code):
# initialize your variable to None
    lat_lng_coords = None
# loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    return lat_lng_coords
    
get_latlng('M9W')

[43.71174000000008, -79.57918134599998]

In [25]:
postal_codes = df2['PostalCode']    
coordins = [ get_latlng(postal_code) for postal_code in postal_codes.tolist() ]

Status code Unknown from https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find: ERROR - HTTPSConnectionPool(host='geocode.arcgis.com', port=443): Read timed out. (read timeout=5.0)
Status code Unknown from https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find: ERROR - HTTPSConnectionPool(host='geocode.arcgis.com', port=443): Read timed out. (read timeout=5.0)


In [26]:
df3 = pd.DataFrame(coordins, columns=['Latitude', 'Longitude'])
df2['Latitude'] = df3['Latitude']
df2['Longitude'] = df3['Longitude']

In [23]:
df2[df2.PostalCode == 'M9W']

,PostalCode,Borough,Neighbourhood
176,M9W,Etobicoke,Northwest


In [28]:
df2

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1A,Not assigned,Not assigned,43.648690,-79.385440
1,M1B,Scarborough,"Malvern, Rouge",43.811525,-79.195517
2,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill",43.785665,-79.158725
3,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765815,-79.175193
4,M1G,Scarborough,Woburn,43.768369,-79.217590
5,M1H,Scarborough,Cedarbrae,43.769688,-79.239440
6,M1J,Scarborough,Scarborough Village,43.743125,-79.231750
7,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.726276,-79.263625
8,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.713054,-79.285055
9,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village ...",43.724235,-79.227925


# Part III: Clustering

In [8]:
import matplotlib.colors as colors
import matplotlib.cm as cm
from sklearn.cluster import KMeans
import re
import json
from geopy.geocoders import Nominatim
import folium

ModuleNotFoundError: No module named 'folium'

In [9]:
address = 'Toronto'


geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, Neighbourhood in zip(df2['Latitude'], df2['Longitude'], df2['Borough'], df2['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, Borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#080807',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


The geograpical coordinate of Toronto are 43.653963, -79.387207.


NameError: name 'folium' is not defined